# Make best_parameters dico file : 

In [10]:
import os 
import json
from tqdm import tqdm

path_work = "/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters"

In [3]:
def make_json(OPTUNA_PATH) : 
    DICO_OPTUNA = {}
    for file in os.listdir(OPTUNA_PATH):
        if file.endswith("json"):
            kl_type = file.split("_")[0]
            with open(f"{OPTUNA_PATH}/{file}", "r") as f:
                best_parameters = json.load(f)
            DICO_OPTUNA[kl_type] = best_parameters
    return DICO_OPTUNA

In [15]:
model_name = { 
"ensemble_20112024_log_optimized_SAGE_ultraF" : "SAGE_uf",
"ensemble_20112024_log_optimized_TropiGAT" : "TropiGAT",
"ensemble_20112024_log_optimized_TropiGAT_ultraF" : "TropiGAT_uf",
"ensemble_20112024_log_optimized_SAGE" : "SAGE"
}

best_para_dico = {}
for rep in tqdm(os.listdir(path_work)) : 
    dico_rep = make_json(f"{path_work}/{rep}")
    best_para_dico[model_name[rep]] = dico_rep
    
with open(f"{path_work}/DAG_models_best_para.json", "w") as fp:
    json.dump(best_para_dico, fp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 440.29it/s]


In [17]:
best_para_dico["TropiGAT"]

{'KL136': {'lr': 0.0041970459851188606,
  'weight_decay': 2.926685033497653e-06,
  'dropout': 0.1918124455994058,
  'att_heads': 3,
  'kl_type': 'KL136'},
 'KL124': {'lr': 0.009297232537363568,
  'weight_decay': 4.123811366547503e-06,
  'dropout': 0.2901572073581546,
  'att_heads': 2,
  'kl_type': 'KL124'},
 'KL61': {'lr': 0.0005272148849251915,
  'weight_decay': 3.3452918946763225e-05,
  'dropout': 0.476098690135296,
  'att_heads': 1,
  'kl_type': 'KL61'},
 'KL107': {'lr': 0.0008482861000659832,
  'weight_decay': 3.148519171507116e-06,
  'dropout': 0.39053809776557713,
  'att_heads': 2,
  'kl_type': 'KL107'},
 'KL67': {'lr': 0.0008341247363255082,
  'weight_decay': 4.274947934461763e-05,
  'dropout': 0.4569116809100993,
  'att_heads': 1,
  'kl_type': 'KL67'},
 'KL48': {'lr': 0.0018953929614386794,
  'weight_decay': 1.0634652408754845e-07,
  'dropout': 0.49255169250086006,
  'att_heads': 3,
  'kl_type': 'KL48'},
 'KL153': {'lr': 0.004784526226360826,
  'weight_decay': 0.000327304616262

# Fixing the ultrafiltration process: 

In [ ]:
import os
import json
import random
import warnings
from collections import Counter
from itertools import product
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import (accuracy_score, f1_score, matthews_corrcoef,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, label_binarize
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import DataLoader, HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GATv2Conv, HeteroConv, to_hetero
from torch_geometric.utils import negative_sampling
from tqdm import tqdm

import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")

# Constants
# **************************************************
ultrafiltration = True
# **************************************************

PATH_WORK = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
DATE = "27_11_2024"

if ultrafiltration == False :
    ENSEMBLE_PATH = f"{PATH_WORK}/train_nn/TropiGAT_ensemble_{DATE}"
    ENSEMBLE_PATH_log = f"{PATH_WORK}/train_nn/TropiGAT_ensemble_{DATE}_log"
    OPTUNA_PATH = f"{PATH_WORK}/train_nn/ensemble_20112024_log_optimized_TropiGAT"
else :
    ENSEMBLE_PATH = f"{PATH_WORK}/train_nn/TropiGAT_ensemble_ultraF_{DATE}"
    ENSEMBLE_PATH_log = f"{PATH_WORK}/train_nn/TropiGAT_ensemble_ultraF_{DATE}_log"
    OPTUNA_PATH = f"{PATH_WORK}/train_nn/ensemble_20112024_log_optimized_TropiGAT_ultraF"
        
os.makedirs(ENSEMBLE_PATH, exist_ok=True)
os.makedirs(ENSEMBLE_PATH_log, exist_ok=True)

DICO_OPTUNA = {}
for file in os.listdir(OPTUNA_PATH):
    if file.endswith("json"):
        kl_type = file.split("_")[0]
        with open(f"{OPTUNA_PATH}/{file}", "r") as f:
            best_parameters = json.load(f)
        DICO_OPTUNA[kl_type] = best_parameters 

def load_and_preprocess_data():
    """Load and preprocess the prophage data."""
    df_info = pd.read_csv(f"{PATH_WORK}/train_nn/TropiGATv2.final_df_v2.tsv", sep="\t", header=0)
    df_info = df_info.drop_duplicates(subset=["Protein_name"])
    
    df_prophages = df_info.drop_duplicates(subset=["Phage"], keep="first")
    dico_prophage_info = {row["Phage"]: {"prophage_strain": row["prophage_id"], "ancestor": row["Infected_ancestor"]} 
                          for _, row in df_prophages.iterrows()}
    
    return df_info, dico_prophage_info

def filter_prophages(df_info, dico_prophage_info):
    """Filter prophages to remove duplicates and ensure diversity."""
    def get_filtered_prophages(prophage):
        combinations = []
        to_exclude = set()
        to_keep = set()
        to_keep.add(prophage)
        df_prophage_group = df_info[
            (df_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & 
            (df_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])
        ]
        if len(df_prophage_group) == 1:
            return df_prophage_group, to_exclude, to_keep
        
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        for prophage_tmp in df_prophage_group["Phage"].unique():
            if prophage_tmp != prophage:
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if depo_set == tmp_depo_set:
                    to_exclude.add(prophage_tmp)
                elif tmp_depo_set not in combinations:
                    to_keep.add(prophage_tmp)
                    combinations.append(tmp_depo_set)
                else:
                    to_exclude.add(prophage_tmp)
        return df_prophage_group, to_exclude, to_keep

    good_prophages = set()
    excluded_prophages = set()

    for prophage in tqdm(dico_prophage_info.keys()):
        if prophage not in excluded_prophages and prophage not in good_prophages:
            _, excluded_members, kept_members = get_filtered_prophages(prophage)
            good_prophages.update(kept_members)
            excluded_prophages.update(excluded_members)

    df_info_filtered = df_info[df_info["Phage"].isin(good_prophages)]
    df_info_final = df_info_filtered[~df_info_filtered["KL_type_LCA"].str.contains("\\|")]

    return df_info_final


def ultrafilter_prophages(df_info):
    """Perform ultra-filtration to remove duplicate prophages within KL types."""
    duplicate_prophage = []
    dico_kltype_duplica = {}

    for kltype in df_info["KL_type_LCA"].unique():
        df_kl = df_info[df_info["KL_type_LCA"] == kltype][["Phage", "Protein_name", "KL_type_LCA", "Infected_ancestor", "index", "seq", "domain_seq"]]
        prophages_tmp_list = df_kl["Phage"].unique().tolist()
        set_sets_depo = []
        duplicated = {}  
        for prophage_tmp in prophages_tmp_list: 
            set_depo = frozenset(df_kl[df_kl["Phage"] == prophage_tmp]["domain_seq"].values)
            for past_set in set_sets_depo:
                if past_set == set_depo:
                    duplicated[past_set] = duplicated.get(past_set, 0) + 1
                    duplicate_prophage.append(prophage_tmp)
                    break
            else:
                set_sets_depo.append(set_depo)
                duplicated[set_depo] = 1
        dico_kltype_duplica[kltype] = duplicated

    df_info_ultrafiltered = df_info[~df_info["Phage"].isin(duplicate_prophage)]
    return df_info_ultrafiltered


def prepare_kltypes(df_info):
    """Prepare KL types for training."""
    df_prophages = df_info.drop_duplicates(subset=["Phage"])
    dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))
    kltypes = [kltype for kltype, count in dico_prophage_count.items() if count >= 10]
    return kltypes, dico_prophage_count



def train_graph(kl_type, graph_baseline, dico_prophage_kltype_associated, df_info, dico_prophage_count):
    """Train the graph neural network for a specific KL type."""
    for seed in range(1, 6):
        if os.path.isfile(f"{ENSEMBLE_PATH_log}/{kl_type}__{seed}__node_classification.{DATE}.log") == False :
            log_file = f"{ENSEMBLE_PATH_log}/{kl_type}__{seed}__node_classification.{DATE}.log"
            with open(log_file, "w") as log_outfile:
                try:
                    n_prophage = dico_prophage_count[kl_type]
                    graph_data_kltype = TropiGAT_graph.build_graph_masking_v2(
                        graph_baseline, dico_prophage_kltype_associated, df_info, 
                        kl_type, 5, 0.7, 0.2, 0.1, seed=seed
                    )

                    model = TropiGAT_models.TropiGAT_small_module(
                        hidden_channels=1280, heads= DICO_OPTUNA[kl_type]["att_heads"], dropout = DICO_OPTUNA[kl_type]["dropout"]
                    )
                    model(graph_data_kltype)

                    optimizer = torch.optim.AdamW(
                        model.parameters(), 
                        lr=DICO_OPTUNA[kl_type]["lr"], 
                        weight_decay=DICO_OPTUNA[kl_type]["weight_decay"]
                    )
                    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, min_lr=1e-6)
                    criterion = torch.nn.BCEWithLogitsLoss()
                    early_stopping = TropiGAT_models.EarlyStopping(
                        patience=100, 
                        verbose=True, 
                        path=f"{ENSEMBLE_PATH}/{kl_type}__{seed}.TropiGATv2.{DATE}.pt", 
                        metric='MCC'
                    )

                    for epoch in range(500):
                        train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer, criterion)
                        if epoch % 5 == 0:
                            test_loss, metrics = TropiGAT_models.evaluate(
                                model, graph_data_kltype, criterion, graph_data_kltype["B1"].test_mask
                            )
                            log_outfile.write(f'Epoch: {epoch}\tTrain Loss: {train_loss:.4f}\t'
                                              f'Test Loss: {test_loss:.4f}\tMCC: {metrics[3]:.4f}\t'
                                              f'AUC: {metrics[5]:.4f}\tAccuracy: {metrics[4]:.4f}\n')
                            scheduler.step(test_loss)
                            early_stopping(metrics[3], model, epoch)
                            if early_stopping.early_stop:
                                log_outfile.write(f"Early stopping at epoch {epoch}\n")
                                break
                    else:
                        torch.save(model, f"{ENSEMBLE_PATH}/{kl_type}__{seed}.TropiGATv2.{DATE}.pt")

                    # Final evaluation
                    model_final = TropiGAT_models.TropiGAT_small_module(
                        hidden_channels=1280, heads = DICO_OPTUNA[kl_type]["att_heads"], dropout = DICO_OPTUNA[kl_type]["dropout"]
                    )
                    model_final.load_state_dict(torch.load(f"{ENSEMBLE_PATH}/{kl_type}__{seed}.TropiGATv2.{DATE}.pt"))
                    eval_loss, metrics = TropiGAT_models.evaluate(
                        model_final, graph_data_kltype, criterion, graph_data_kltype["B1"].eval_mask
                    )

                    with open(f"{ENSEMBLE_PATH_log}/Metric_Report.{DATE}.tsv", "a+") as metric_outfile:
                        metric_outfile.write(f"{kl_type}__{seed}\t{n_prophage}\t"
                                             f"{metrics[0]:.4f}\t{metrics[1]:.4f}\t{metrics[2]:.4f}\t"
                                             f"{metrics[3]:.4f}\t{metrics[4]:.4f}\t{metrics[5]:.4f}\n")

                    log_outfile.write(f"Final evaluation:\n"
                                      f"F1 Score: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, "
                                      f"Recall: {metrics[2]:.4f}, MCC: {metrics[3]:.4f}, "
                                      f"Accuracy: {metrics[4]:.4f}, AUC: {metrics[5]:.4f}")

                except Exception as e:
                    log_outfile.write(f"Error occurred: {str(e)}")
                    with open(f"{ENSEMBLE_PATH_log}/Metric_Report.{DATE}.tsv", "a+") as metric_outfile:
                        metric_outfile.write(f"{kl_type}__{seed}\t{n_prophage}\t***Error***\n")

def main():
    """Main function to orchestrate the TropiGAT workflow."""
    df_info, dico_prophage_info = load_and_preprocess_data()
    if ultrafiltration == True :
        df_info_final = ultrafilter_prophages(filter_prophages(df_info, dico_prophage_info))
    else :
        df_info_final = filter_prophages(df_info, dico_prophage_info)
    
    kltypes, dico_prophage_count = prepare_kltypes(df_info_final)
    graph_baseline, dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(df_info_final)
    
    with ThreadPool(5) as p:
        p.starmap(train_graph, [(kl_type, graph_baseline, dico_prophage_kltype_associated, df_info_final, dico_prophage_count) for kl_type in kltypes])

if __name__ == '__main__':
    main()


# Get the log files with the best parameters : 


In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_20112024_log_optimized_TropiGAT \
/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters


rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_20112024_log_optimized_TropiGAT_ultraF \
/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters


rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_20112024_log_optimized_SAGE \
/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters


rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_20112024_log_optimized_SAGE_ultraF \
/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters


In [9]:
import os
import json

path_json = "/media/concha-eloko/Linux/PPT_clean/trainer_best_parameters/ensemble_20112024_log_optimized_TropiGAT"
OPTUNA_PATH = path_json


with open(f"{path_json}/KL136_optuna_best_params.json", "r") as f:  # Use context manager to open the file
    best_parameters = json.load(f)

In [10]:
DICO_OPTUNA = {}
for file in os.listdir(OPTUNA_PATH):
    if file.endswith("json"):
        kl_type = file.split("_")[0]
        with open(f"{OPTUNA_PATH}/{file}", "r") as f:
            best_parameters = json.load(f)
        DICO_OPTUNA[kl_type] = best_parameters


# Other

In [1]:
import os 
import pandas as pd
from collections import Counter


path_work = "/media/concha-eloko/Linux/PPT_clean"

bacteria_data_df = pd.read_csv(f"{path_work}/results_kleborate_count.tsv", sep = "\t", header = 0)

DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_amb = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]

#DF_info_lvl_0_filter1 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

In [3]:
import os
import random
import warnings
from collections import Counter
from multiprocessing.pool import ThreadPool
import json
import math
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import (accuracy_score, f1_score, matthews_corrcoef,
                             precision_score, recall_score, roc_auc_score)
import optuna
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GATv2Conv, HeteroConv
from tqdm import tqdm

/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/libpyg.so: undefined symbol: _ZN3c1010Dispatcher17runRecordFunctionERN2at14RecordFunctionESt17reference_wrapperIKNS_14FunctionSchemaEENS_11DispatchKeyE
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /m

In [ ]:
import TropiGAT_graph
import TropiGAT_models
import 

In [2]:
! python -m pip install optuna


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 4.0 MB/s eta 0:00:002.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 190.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 91.8 MB/s eta 0:00:00


# Not sure what happends there :

In [ ]:
import os 
import pandas as pd
from collections import Counter


path_work = "/media/concha-eloko/Linux/PPT_clean"

bacteria_data_df = pd.read_csv(f"{path_work}/results_kleborate_count.tsv", sep = "\t", header = 0)

DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_amb = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]

#DF_info_lvl_0_filter1 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

In [12]:
path_prophage = "/media/concha-eloko/Linux/prediction_depolymerase_tropism/prophage_work/prophage_prediction"
f_labels=["Prophage name","K-serotype monophyletic group","Id monophyletic group","Number of clades","Number of leafs","Number of new ancestors","Number of k-type swap","Nodes k-types","Nodes k-types all"]

df_prophages_1 = pd.read_csv(f"{path_prophage}/prophage_data.clusters_80.phageboost_70.final.tsv", sep="\t", names =f_labels) 
df_prophages_2 = pd.read_csv(f"{path_work}/prophage_data.clusters_80.phageboost_70.2504.tsv", sep="\t", names =f_labels, skiprows=1) 



In [7]:
df_prophages_1

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
0,GCF_020615655.1__EMCPJNLC_9,KL64,n4043,1,2,1,0,None,None
1,GCF_021213885.1__GOKIIELC_105,KL19,n28462846,4,5,4,0,KL19,"KL19,KL19,KL19,KL19"
2,GCF_003666725.1__MDOIKHEP_22,KL64,n2838,1,2,1,0,None,None
3,GCF_002261745.1__NCJKIGBL_29,KL47,n23482348,2,3,2,0,KL47,"KL47,KL47"
4,GCF_003262135.1__LHIFPDKB_94,KL64,n26072607,3,4,3,0,KL64,"KL64,KL64,KL64"
...,...,...,...,...,...,...,...,...,...
60064,GCF_900516735.1__KGBILJNM_66,KL54,GCF_900516735.1,0,1,0,0,None,None
60065,GCF_020479585.1__CKJNGDMH_2,KL49,n54935493,1,2,1,0,None,None
60066,GCF_900510465.1__KNLMOLHD_60,KL14,n4610,2,3,1,0,None,None
60067,GCF_902162525.1__BAKPCPAK_6,KL164,GCF_902162525.1,0,1,0,0,None,None


In [17]:
df_prophages_2_amb = df_prophages_2[df_prophages_2["K-serotype monophyletic group"].str.contains("\\|")]
df_prophages_2_amb

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
167,GCF_014288695.1__phage4.fasta,KL122|KL25|KL3|KL125|KL24|KL2|KL31|KL30|KL35|K...,n5891,1,2,1,0,None,None
170,GCF_019930245.1__phage16.fasta,KL60|KL110|KL142,n43774377,5,6,1,0,None,None
250,GCF_019038845.1__phage7.fasta,KL122|KL128|KL25|KL43|KL61|KL3|KL151|KL53|KL10...,n58915891,3,4,1,0,None,None
255,GCF_014286295.1__phage3.fasta,KL122|KL43|KL25|KL61|KL3|KL151|KL48|KL53|KL125...,n54705470,1,2,1,0,None,None
259,GCF_003692695.1__phage23.fasta,KL122|KL43|KL25|KL3|KL125|KL24|KL2|KL31|KL140|...,n2021,3,4,1,0,None,None
...,...,...,...,...,...,...,...,...,...
75565,GCF_900507595.1__phage7.fasta,KL15|KL30,49.5/100,34,35,1,0,None,None
76235,GCF_016635675.1__phage0.fasta,KL30|KL133|KL11,n5931,13,14,3,1,"KL30|KL64|KL14|KL133|KL10|KL47|KL11|KL39,KL47","KL30|KL64|KL14|KL133|KL10|KL47|KL11|KL39,KL11|..."
76249,GCF_900507185.1__phage10.fasta,KL28|KL61|KL110,n13911391,3,4,1,0,None,None
76529,GCF_020178015.1__phage0.fasta,KL43|KL25|KL3|KL136|KL125|KL24|KL116|KL2|KL30|...,n492,1,2,1,0,None,None


In [20]:
df_prophages_2

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
0,GCF_017310325.1__phage26.fasta,KL15,n3842,1,2,1,0,None,None
1,GCF_017311795.1__phage12.fasta,KL111,n40554055,4,5,2,0,KL111,"KL111,KL111"
2,GCF_922827125.1__phage19.fasta,KL102,76.3/100,3,4,2,0,KL102,"KL102,KL102"
3,GCF_014840315.1__phage19.fasta,KL112,n10661066,3,4,1,0,None,None
4,GCF_003225795.1__phage33.fasta,KL30,n47034703,1,2,1,0,None,None
...,...,...,...,...,...,...,...,...,...
77798,GCF_905127585.1__phage2.fasta,KL64,GCF_905127585.1,0,1,0,0,None,None
77799,GCF_002187935.3__phage23.fasta,KL14,GCF_002187935.3,0,1,0,0,None,None
77800,GCF_017898175.1__phage24.fasta,KL64,GCF_017898175.1,0,1,0,0,None,None
77801,GCF_003955455.1__phage28.fasta,KL14,GCF_003955455.1,0,1,0,0,None,None


In [19]:
max(df_prophages_2["Number of new ancestors"])

72